# Building a docker container with the code for training our classifier
In this exercise we'll create a Docker image that will have the required code for training and deploying a ML model. In this particular example, we'll use scikit-learn (https://scikit-learn.org/) and the **Random Forest Tree** implementation of that library to train a flower classifier. The dataset used in this experiment is a toy dataset called Iris (http://archive.ics.uci.edu/ml/datasets/iris). The clallenge itself is very basic, so you can focus on the mechanics and the features of this automated environment.

A first pipeline will be executed at the end of this exercise, automatically. It will get the assets you'll push to a Git repo, build this image and push it to ECR, a docker image repository, used by SageMaker.

> **Question**: Why would I create a Scikit-learn container from scratch if SageMaker already offerst one (https://docs.aws.amazon.com/sagemaker/latest/dg/sklearn.html).  
> **Answer**: This is an exercise and the idea here is also to show you how you can create your own container. In a real-life scenario, the best approach is to use the native container offered by SageMaker.

## PART 1 - Creating the assets required to build a docker image

### Lets start by creating a Dockerfile
Just pay attention to the packages we'll install in our container. Here, we'll use **SageMaker Inference Toolkit** (https://github.com/aws/sagemaker-inference-toolkit) to prepare the container for serving our model (or exposing our model as a webservice that can be called through an api call). We'll also create our training script, as you will see ahead 

In [ ]:
%%writefile Dockerfile
FROM python:3.7-buster

# Set a docker label to advertise multi-model support on the container
LABEL com.amazonaws.sagemaker.capabilities.multi-models=false
# Set a docker label to enable container to use SAGEMAKER_BIND_TO_PORT environment variable if present
LABEL com.amazonaws.sagemaker.capabilities.accept-bind-to-port=true

RUN apt-get update -y && apt-get -y install --no-install-recommends default-jdk
RUN rm -rf /var/lib/apt/lists/*

RUN pip --no-cache-dir install multi-model-server sagemaker-inference
RUN pip --no-cache-dir install pandas numpy scipy scikit-learn

RUN mkdir -p /opt/program
RUN mkdir -p /opt/ml/model

ENV PYTHONUNBUFFERED=TRUE
ENV PYTHONDONTWRITEBYTECODE=TRUE
ENV PATH="/opt/program:${PATH}"

COPY inference_handler.py /opt/program
COPY training.py /opt/program

WORKDIR /opt/program

EXPOSE 8080
ENTRYPOINT ["python", "inference_handler.py"]

Ok. Lets then create the handler. The **Inference Handler** is how we use the Inference Toolkit to encapsulate our code and expose it as a SageMaker container.

In [ ]:
%%writefile inference_handler.py
import os
import sys
import training
import joblib
from sagemaker_inference.default_inference_handler import DefaultInferenceHandler
from sagemaker_inference.default_handler_service import DefaultHandlerService
from sagemaker_inference import content_types, errors, transformer, model_server, encoder, decoder

class InferenceHandler(DefaultInferenceHandler):
    ## Loads the model from the disk
    def default_model_fn(self, model_dir):
        model_filename = os.path.join(model_dir, 'iris_model.pkl')
        return joblib.load(open(model_filename, 'rb'))
    
    ## Parse and check the format of the input data
    def default_input_fn(self, input_data, content_type):
        if content_type != 'text/csv':
            raise Exception('Invalid content-type: %s' % content_type)
        return decoder.decode(input_data, content_type).reshape(1,-1)
    
    ## Run our model and do the prediction
    def default_predict_fn(self, payload, model):
        return model.predict( payload ).tolist()
    
    ## Gets the prediction output and format it to be returned to the user
    def default_output_fn(self, prediction, accept):
        if accept != 'text/csv':
            raise Exception('Invalid accept: %s' % accept)
        return encoder.encode(prediction, accept)

class HandlerService(DefaultHandlerService):
    def __init__(self):
        op = transformer.Transformer(default_inference_handler=InferenceHandler())
        super(HandlerService, self).__init__(transformer=op)
        
_service = HandlerService()
def handle(data, context):
    if not _service._service._initialized:
        _service.initialize(context)

    if data is None:
        return None

    return _service.handle(data, context)

if __name__ == '__main__':
    if len(sys.argv) < 2 or ( not sys.argv[1] in [ "serve", "train"] ):
        raise Exception("Invalid argument: you must inform 'train' for training mode or 'serve' predicting mode") 
        
    if sys.argv[1] == "train":
        training.start()
    else:
        model_server.start_model_server(handler_service="/opt/program/inference_handler.py:handle")

### Now, we need to create our training script. SageMaker will use it for training/saving our model
You can see that this is a very basic usage of Scikit-Learn.  
Also, you'll see that **/opt/ml** is the directory used by SageMaker to "communicate" with your code.

In [ ]:
%%writefile training.py
import os
import pandas as pd
import re
import joblib
import json
from sklearn.ensemble import RandomForestClassifier

# This directory is the communication channel between Sagemaker and your container
prefix = '/opt/ml'

# Here, Sagemaker will store the dataset copyied from S3
input_path = os.path.join(prefix, 'input/data')
# If something bad happens, write a failure file with the error messages and store here
output_path = os.path.join(prefix, 'output')
# Everything you store here will be packed into a .tar.gz by Sagemaker and store into S3
model_path = os.path.join(prefix, 'model')
# This is the hyperparameters you will send to your algorithms through the Estimator
param_path = os.path.join(prefix, 'input/config/hyperparameters.json')

def start():
    print("Training mode")

    try:
        training_path = os.path.join(input_path, "train")
        validation_path = os.path.join(input_path, "validation")

        hyperparameters = {}
        # Read in any hyperparameters that the user passed with the training job
        with open(param_path, 'r') as tc:
            is_float = re.compile(r'^\d+(?:\.\d+)$')
            is_integer = re.compile(r'^\d+$')
            for key,value in json.load(tc).items():
                # workaround to convert numbers from string
                if is_float.match(value) is not None:
                    value = float(value)
                elif is_integer.match(value) is not None:
                    value = int(value)
                hyperparameters[key] = value

        # Take the set of files and read them all into a single pandas dataframe
        train_files = [ os.path.join(training_path, file) for file in os.listdir(training_path) ]
        validation_files = [ os.path.join(validation_path, file) for file in os.listdir(validation_path) ]
        if len(train_files) == 0 or len(validation_files) == 0:
            raise ValueError(('There are no files in {} or {}.\\n' +
                              'This usually indicates that the channel ({} or {}) was incorrectly specified,\\n' +
                              'the data specification in S3 was incorrectly specified or the role specified\\n' +
                              'does not have permission to access the data.').format(
                training_path, 'train',validation_path, 'validation'))
        
        raw_data = [ pd.read_csv(file, sep=',', header=None ) for file in train_files ]
        train_data = pd.concat(raw_data)
        raw_data = [ pd.read_csv(file, sep=',', header=None ) for file in validation_files ]
        validation_data = pd.concat(raw_data)
        
        # labels are in the first column
        Y_train = train_data.iloc[:,0]
        X_train = train_data.iloc[:,1:]
        Y_validation = validation_data.iloc[:,0]
        X_validation = validation_data.iloc[:,1:]

        print("Training the classifier")
        model = RandomForestClassifier()
        hyperparameters['verbose'] = 1 # show all logs
        model.set_params(**hyperparameters)
        model.fit(X_train, Y_train)
        print("Score: {}".format( model.score(X_validation, Y_validation)) )
        joblib.dump(model, open(os.path.join(model_path, 'iris_model.pkl'), 'wb'))
    
    except Exception as e:
        # Write out an error file. This will be returned as the failureReason in the
        # DescribeTrainingJob result.
        trc = traceback.format_exc()
        with open(os.path.join(output_path, 'failure'), 'w') as s:
            s.write('Exception during training: ' + str(e) + '\\n' + trc)
            
        # Printing this causes the exception to be in the training job logs, as well.
        print('Exception during training: ' + str(e) + '\\n' + trc, file=sys.stderr)
        
        # A non-zero exit code causes the training job to be marked as Failed.
        sys.exit(255)

### Finally, let's create the buildspec
This file will be used by CodeBuild for creating our Container image.  
With this file, CodeBuild will run the "docker build" command, using the assets we created above, and deploy the image to the Registry.  
As you can see, each command is a bash command that will be executed from inside a Linux Container.

In [ ]:
%%writefile buildspec.yml
version: 0.2

phases:
  install:
    runtime-versions:
      docker: 18

  pre_build:
    commands:
      - echo Logging in to Amazon ECR...
      - $(aws ecr get-login --no-include-email --region $AWS_DEFAULT_REGION)
      - docker pull $AWS_ACCOUNT_ID.dkr.ecr.$AWS_DEFAULT_REGION.amazonaws.com/scikit-base:latest
      - docker tag $AWS_ACCOUNT_ID.dkr.ecr.$AWS_DEFAULT_REGION.amazonaws.com/scikit-base:latest scikit-base:latest
  build:
    commands:
      - echo Build started on `date`
      - echo Building the Docker image...
      - docker build -t $IMAGE_REPO_NAME:$IMAGE_TAG .
      - docker tag $IMAGE_REPO_NAME:$IMAGE_TAG $AWS_ACCOUNT_ID.dkr.ecr.$AWS_DEFAULT_REGION.amazonaws.com/$IMAGE_REPO_NAME:$IMAGE_TAG

  post_build:
    commands:
      - echo Build completed on `date`
      - echo Pushing the Docker image...
      - echo docker push $AWS_ACCOUNT_ID.dkr.ecr.$AWS_DEFAULT_REGION.amazonaws.com/$IMAGE_REPO_NAME:$IMAGE_TAG
      - docker push $AWS_ACCOUNT_ID.dkr.ecr.$AWS_DEFAULT_REGION.amazonaws.com/$IMAGE_REPO_NAME:$IMAGE_TAG
      - echo $AWS_ACCOUNT_ID.dkr.ecr.$AWS_DEFAULT_REGION.amazonaws.com/$IMAGE_REPO_NAME:$IMAGE_TAG > image.url
      - echo Done
artifacts:
  files:
    - image.url
  name: image_url
  discard-paths: yes

## PART 2 - Local Test: Let's build the image locally and do some tests
### Building the image locally, first
Each SageMaker Jupyter Notebook already has a **docker** envorinment pre-installed. So we can play with Docker containers just using the same environment.

In [ ]:
!docker build -f Dockerfile -t iris_model:1.0 .

### Now that we have the algorithm image we can run it to train/deploy a model

First, let's define some hyperparameters

In [ ]:
hyperparameters = {
    "max_depth": 11,
    "n_jobs": 5,
    "n_estimators": 120
}

In [ ]:
import json
!mkdir -p input/config

hyperparameters = dict({key: str(values) for key, values in hyperparameters.items()})
with open('input/config/hyperparameters.json', 'w') as f:
    f.write(json.dumps(hyperparameters))
    f.flush()
    f.close()

### Then, we need to prepare the dataset
You'll see that we're splitting the dataset into training and validation and also saving these two subsets of the dataset into csv files. These files will be then uploaded to an S3 Bucket and shared with SageMaker.

In [ ]:
!mkdir -p input/data/train
!mkdir -p input/data/validation

import pandas as pd
import numpy as np

from sklearn import datasets
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()

dataset = np.insert(iris.data, 0, iris.target,axis=1)

df = pd.DataFrame(data=dataset, columns=['iris_id'] + iris.feature_names)
X = df.iloc[:,1:]
y = df.iloc[:,0]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33, random_state=42)

train_df = X_train.copy()
train_df.insert(0, 'iris_id', y_train)
train_df.to_csv('input/data/train/training.csv', sep=',', header=None, index=None)

val_df = X_val.copy()
val_df.insert(0, 'iris_id', y_val)
val_df.to_csv('input/data/validation/validation.csv', sep=',', header=None, index=None)

df.head()

### Just a basic local test, using the local Docker daemon

In [ ]:
%%time
!rm -rf model/
!mkdir -p model
print( "Training ...")
!docker run --rm --name 'my_model' \
    -v "$PWD/model:/opt/ml/model" \
    -v "$PWD/input:/opt/ml/input" iris_model:1.0 train

### This is the serving test. It simulates an Endpoint exposed by Sagemaker

After you execute the next cell, this Jupyter notebook will freeze. A webservice will be exposed at the port 8080. 

In [ ]:
!docker run --rm --name 'my_model' \
    -p 8080:8080 \
    -v "$PWD/model:/opt/ml/model" \
    -v "$PWD/input:/opt/ml/input" iris_model:1.0 serve

> While the above cell is running, click here [TEST NOTEBOOK](02_Testing%20our%20local%20model%20server.ipynb) to run some tests.

> After you finish the tests, press **STOP**

## PART 3 - Integrated Test: Everything seems ok, now it's time to put all together

We'll start by running a local **CodeBuild** test, to check the buildspec and also deploy this image into the container registry. Remember that SageMaker will only see images published to ECR.


In [ ]:
import boto3

sts_client = boto3.client("sts")
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
credentials = session.get_credentials()
credentials = credentials.get_frozen_credentials()

repo_name='iris-model'
image_tag='test'

In [ ]:
!rm -rf tests && mkdir -p tests
!cp inference_handler.py training.py Dockerfile buildspec.yml tests/
with open('tests/vars.env', 'w') as f:
    f.write("AWS_ACCOUNT_ID=%s\n" % account_id)
    f.write("IMAGE_TAG=%s\n" % image_tag)
    f.write("IMAGE_REPO_NAME=%s\n" % repo_name)
    f.write("AWS_DEFAULT_REGION=%s\n" % region)
    f.write("AWS_ACCESS_KEY_ID=%s\n" % credentials.access_key)
    f.write("AWS_SECRET_ACCESS_KEY=%s\n" % credentials.secret_key)
    f.write("AWS_SESSION_TOKEN=%s\n" % credentials.token )
    f.close()

!cat tests/vars.env

In [ ]:
%%time

!/tmp/aws-codebuild/local_builds/codebuild_build.sh \
    -a "$PWD/tests/output" \
    -s "$PWD/tests" \
    -i "samirsouza/aws-codebuild-standard:3.0" \
    -e "$PWD/tests/vars.env" \
    -c

> Now that we have an image deployed in the ECR repo we can also run some local tests using the SageMaker Estimator.

> Click on this [TEST NOTEBOOK](03_Testing%20the%20container%20using%20SageMaker%20Estimator.ipynb) to run some tests.

> After you finishing the tests, come back to **this notebook** to push the assets to the Git Repo


## PART 4 - Let's push all the assets to the Git Repo connected to the Build pipeline
There is a CodePipeine configured to keep listeining to this Git Repo and start a new Building process with CodeBuild.

In [ ]:
%%bash
cd ../../../mlops
git checkout iris_model
cp $OLDPWD/buildspec.yml $OLDPWD/inference_handler.py $OLDPWD/training.py $OLDPWD/Dockerfile .

git add --all
git commit -a -m " - files for building an iris model image"
git push

> Alright, now open the AWS console and go to the **CodePipeline** dashboard. Look for a pipeline called **mlops-iris-model**. This pipeline will deploy the final image to an ECR repo. When this process finishes, open the **Elastic Compute Registry** dashboard, in the AWS console, and check if you have an image called **iris-model:latest**. If yes, you can go to the next exercise. If not, wait a little more.